ASAP CRN Metadata compilation

# Team Lee (Biederer). ASAP CRN Metadata construction


This is a bulkRNAseq dataset which was originally transfered along with the scRNAseq dataset.  The metadata should be identical, save the details of the bulkRNAseq assay compared to the sc/snRNAseq previously platformed.



15 Sept 2024
Andy Henrie




In [1]:
import pandas as pd
from pathlib import Path
import os, sys

sys.path.append(os.path.abspath((os.path.join(os.getcwd(), 'src/crn_utils'))))

from util import read_CDE, NULL, prep_table, read_meta_table
from validate import validate_table, ReportCollector
from update_schema import v1_to_v2, v2_to_v3_PMDBS, create_upload_medadata_package
from checksums import extract_md5_from_details2

%load_ext autoreload
%autoreload 2



Streamlit NOT successfully imported


In [2]:
!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-lee/fastqs/bulk_MFG/*.gz" > lee_bulk_hexhash.log


using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [3]:


bucket_files_md5 = extract_md5_from_details2("lee_bulk_hexhash.log")


In [4]:
df = pd.DataFrame(bucket_files_md5.items(), columns=["filename", "md5"])


In [5]:
df['id'] = df['filename'].str.split("_").str[0]

In [6]:
df['subject_id'] = df['id'].apply(lambda x: f"{x[-2:]}_{x[:-2]}")
df.head()

,filename,md5,id,subject_id
0,0009PD_MFG_bulk_L000_R1_001.fastq.gz,c9d64ec5b02de6b09a45dd7ac6610854,0009PD,PD_0009
1,0009PD_MFG_bulk_L000_R2_001.fastq.gz,43a19a5fafd6892a0726660a560c4993,0009PD,PD_0009
2,0348PD_MFG_bulk_L000_R1_001.fastq.gz,0ad336467cb40bfa3f6e7f582ce53203,0348PD,PD_0348
3,0348PD_MFG_bulk_L000_R2_001.fastq.gz,8b6c00b59f2f0e08c053cd437a17584d,0348PD,PD_0348
4,0413PD_MFG_bulk_L000_R1_001.fastq.gz,a1e499ed8fd73cc2ae75835792ea23d0,0413PD,PD_0413


In [7]:
df['sample_id'] = "MFG_" + df['subject_id']
df.head()

BULK_FILES = df.copy()

### Team Lee

## CDEs
load the relavent CDEs

In [8]:
schema_version = "v1"
schema_path = Path.home() / "Projects/ASAP/crn-utils/resource/CDE"
CDEv1 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v2.1"
CDEv2 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v3.0"
CDEv3 = read_CDE(schema_version, local_path=schema_path)

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v1.csv
read local file
metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v2.1.csv
read local file
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v3.0.csv
read local file


## Clean v3 Table


In [9]:
## convert 
team = "lee"
collection = "bulk_pmdbs"

root_path = Path.home() / ("Projects/ASAP/data/teams")

metadata_path = root_path / f"{team}/{collection}/metadata/"
v3_path = metadata_path / "v3"
og_path = metadata_path / "og"  # this data is a copy of the lee/sn_pmdbs/metadata/v3

v3_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']

in_tables = [table_name for table_name in v3_meta_tables if f"{table_name}.csv" in os.listdir(og_path)]


In [10]:
og_tables = {}
for table_name in in_tables:
    df = read_meta_table(f"{og_path}/{table_name}.csv")
    og_tables[table_name] = df
    

In [11]:
og_tables.keys()

dict_keys(['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq'])

In [12]:
og_tables['STUDY']

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,number_samples,sample_types,types_of_samples,DUA_version,metadata_tables,PI_ORCID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-LEE,Bras,Is senescence a component of human PD and does...,sc_pmdbs,Human snRNA-seq PD Senesence Jose Bras Team Lee,Characterize the neuropathological progression...,"Jose, Bras",jose.bras@vai.org,"Lee, L, Marshall ; Kimberly, E, Paquette ; Kai...",Kaitlyn E Westra,...,75,hippocampus; middle frontal gyrus; substantia ...,human PD and control postmortem brains,unsure,"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DA...",NA,NA,NA(raw data),v1_20241016,NA


In [13]:
CDE = CDEv3
dfs = {}
for table,df in og_tables.items():
    if table == "ASSAY_bulkRNAseq":
        table = "ASSAY_RNAseq"

    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    dfs[table] = full_table

recoding number_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **7 Fields with empty (NULL) values:**

	- submitter_email: 1/1 empty rows (REQUIRED)

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_ORCID: 1/1 empty rows (OPTIONAL)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- primary_diagnosis_text: 23/25 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding replicate_count as int
recoding repeated_sample as int
All required fields are present in *SAMPLE* table.


In [14]:
STUDY = dfs['STUDY']
PROTOCOL = dfs['PROTOCOL']
SUBJECT = dfs['SUBJECT']
SAMPLE = dfs['SAMPLE']
DATA = dfs['DATA']
CLINPATH = dfs['CLINPATH']
PMDBS = dfs['PMDBS']
ASSAY_RNAseq = dfs['ASSAY_RNAseq']
CONDITION = dfs['CONDITION']

In [15]:
# rationalize the team_dataset_id
STUDY['team_dataset_id'] = collection
STUDY['metadata_tables'] = f"{v3_meta_tables}"

In [16]:
STUDY

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,number_samples,sample_types,types_of_samples,DUA_version,metadata_tables,PI_ORCID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-LEE,Bras,Is senescence a component of human PD and does...,bulk_pmdbs,Human snRNA-seq PD Senesence Jose Bras Team Lee,Characterize the neuropathological progression...,"Jose, Bras",jose.bras@vai.org,"Lee, L, Marshall ; Kimberly, E, Paquette ; Kai...",Kaitlyn E Westra,...,75,hippocampus; middle frontal gyrus; substantia ...,human PD and control postmortem brains,unsure,"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DA...",NA,NA,NA(raw data),v1_20241016,NA


In [17]:
# fix ASSAY_RNAseq  
ASSAY_RNAseq['technology'] = "Bulk"
ASSAY_RNAseq.head()

,sample_id,tissue,technology,omic,RIN,molecular_source,input_cell_count,assay,sequencing_end,sequencing_length,sequencing_instrument
0,MFG_HC_1225,Brain,Bulk,RNA,NA,PolyA RNA,9106,v1,Paired-end,100,Illumina NovaSeq 6000
1,HIP_HC_1225,Brain,Bulk,RNA,NA,PolyA RNA,6533,v1,Paired-end,100,Illumina NovaSeq 6000
2,SN_HC_1225,Brain,Bulk,RNA,NA,PolyA RNA,989,v1,Paired-end,100,Illumina NovaSeq 6000
3,MFG_HC_0602,Brain,Bulk,RNA,NA,PolyA RNA,2675,v1,Paired-end,100,Illumina NovaSeq 6000
4,HIP_HC_0602,Brain,Bulk,RNA,NA,PolyA RNA,2954,v1,Paired-end,100,Illumina NovaSeq 6000



ASSAY:
>> VAI’s Genomics Core (RRID:SCR_022913; they sequenced this data for us), and I can make some inferences after looking through it. From what I can see, this was likely sequenced on the NovaSeq 6000, with a run length of 2x100bp, using stranded total RNA, with a target genome coverage of 30M reads

In [18]:
DATA.head()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,adjustment,content,header,annotation,configuration_file
0,MFG_HC_1225,rep1,1,0,BATCH_4,fastq,MFGHC1225_S9_L001_R1_001.fastq.gz,Raw sequencing data,9977258e598d6a52130c29c71aef6925,Raw,Reads,NA,NA,NA(raw data)
1,MFG_HC_1225,rep1,1,0,BATCH_4,fastq,MFGHC1225_S9_L001_R2_001.fastq.gz,Raw sequencing data,fe2cf93257801227b7072a4fb7d18792,Raw,Reads,NA,NA,NA(raw data)
2,MFG_HC_0602,rep1,1,0,BATCH_4,fastq,MFGHC0602_S2_L001_R1_001.fastq.gz,Raw sequencing data,110ca4864cf6938faca67567bebfb6cc,Raw,Reads,NA,NA,NA(raw data)
3,MFG_HC_0602,rep1,1,0,BATCH_4,fastq,MFGHC0602_S2_L001_R2_001.fastq.gz,Raw sequencing data,0dcc67217e43ab53bae0d0676f9bfe8b,Raw,Reads,NA,NA,NA(raw data)
4,MFG_PD_0009,rep1,1,0,BATCH_4,fastq,MFGPD0009_S3_L001_R1_001.fastq.gz,Raw sequencing data,a2608d0bd192333b0076d7091c1c50ea,Raw,Reads,NA,NA,NA(raw data)


From here we should be able to infer the samples and subjects to subset to


In [19]:
SAMPLE.head()

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,organism,tissue,assay_type,...,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,assay_ontology_term_id,donor_id,pm_PH,cell_type_ontology_term_id,source_RIN,DV200,suspension_type
0,MFG_HC_1225,HC_1225,12-25,rep1,1,0,BATCH_4,NA,Brain,NA,...,NA,PATO:0000461,UBERON:0002702,EFO:0030004,NA,NA,NA(multiple),NA,NA,nucleus
1,HIP_HC_1225,HC_1225,12-25,rep1,1,0,BATCH_9,NA,Brain,NA,...,NA,PATO:0000461,UBERON:0002702,EFO:0030004,NA,NA,NA(multiple),NA,NA,nucleus
2,SN_HC_1225,HC_1225,12-25,rep1,1,0,BATCH_7,NA,Brain,NA,...,NA,PATO:0000461,UBERON:0002702,EFO:0030004,NA,NA,NA(multiple),NA,NA,nucleus
3,MFG_HC_0602,HC_0602,06-02,rep1,1,0,BATCH_4,NA,Brain,NA,...,NA,PATO:0000461,UBERON:0002702,EFO:0030004,NA,NA,NA(multiple),NA,NA,nucleus
4,HIP_HC_0602,HC_0602,06-02,rep1,1,0,BATCH_9,NA,Brain,NA,...,NA,PATO:0000461,UBERON:0002702,EFO:0030004,NA,NA,NA(multiple),NA,NA,nucleus


In [20]:
SUBJECT.head()

,subject_id,source_subject_id,biobank_name,sex,age_at_collection,race,primary_diagnosis,primary_diagnosis_text
0,HC_1225,12-25,Banner Sun Health Research Institute,Male,80.0,White,No PD nor other neurological disorder,NA
1,HC_0602,06-02,Banner Sun Health Research Institute,Male,84.0,White,Other neurological disorder,Mild Cognitive Impairment
2,PD_0009,00-09,Banner Sun Health Research Institute,Male,64.0,White,Idiopathic PD,NA
3,PD_1921,19-21,Banner Sun Health Research Institute,Male,82.0,White,Idiopathic PD,NA
4,PD_2058,20-58,Banner Sun Health Research Institute,Male,87.0,White,Idiopathic PD,NA


In [21]:
BULK_FILES.head()

,filename,md5,id,subject_id,sample_id
0,0009PD_MFG_bulk_L000_R1_001.fastq.gz,c9d64ec5b02de6b09a45dd7ac6610854,0009PD,PD_0009,MFG_PD_0009
1,0009PD_MFG_bulk_L000_R2_001.fastq.gz,43a19a5fafd6892a0726660a560c4993,0009PD,PD_0009,MFG_PD_0009
2,0348PD_MFG_bulk_L000_R1_001.fastq.gz,0ad336467cb40bfa3f6e7f582ce53203,0348PD,PD_0348,MFG_PD_0348
3,0348PD_MFG_bulk_L000_R2_001.fastq.gz,8b6c00b59f2f0e08c053cd437a17584d,0348PD,PD_0348,MFG_PD_0348
4,0413PD_MFG_bulk_L000_R1_001.fastq.gz,a1e499ed8fd73cc2ae75835792ea23d0,0413PD,PD_0413,MFG_PD_0413


In [22]:

bulk_samples = set(BULK_FILES['sample_id'].unique())
bulk_subjects = set(BULK_FILES['subject_id'].unique())


In [23]:
# SUBJECT subset by subject
SUBJECT_ = SUBJECT[SUBJECT['subject_id'].isin(bulk_subjects)]
SUBJECT_.head()
# CLINPATH subset by subject
CLINPATH_ = CLINPATH[CLINPATH['subject_id'].isin(bulk_subjects)]


# PMDDBS subset by sample
PMDBS_ = PMDBS[PMDBS['sample_id'].isin(bulk_samples)]

# SAMPLE subset by sample
SAMPLE_ = SAMPLE[SAMPLE['sample_id'].isin(bulk_samples)]

# ASSAY_RNAseq subset by sample
ASSAY_RNAseq_ = ASSAY_RNAseq[ASSAY_RNAseq['sample_id'].isin(bulk_samples)]


In [24]:

# DATA suubset by sample

DATA_ = DATA[DATA['sample_id'].isin(bulk_samples)]

DATA_.head()



,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,adjustment,content,header,annotation,configuration_file
0,MFG_HC_1225,rep1,1,0,BATCH_4,fastq,MFGHC1225_S9_L001_R1_001.fastq.gz,Raw sequencing data,9977258e598d6a52130c29c71aef6925,Raw,Reads,NA,NA,NA(raw data)
1,MFG_HC_1225,rep1,1,0,BATCH_4,fastq,MFGHC1225_S9_L001_R2_001.fastq.gz,Raw sequencing data,fe2cf93257801227b7072a4fb7d18792,Raw,Reads,NA,NA,NA(raw data)
2,MFG_HC_0602,rep1,1,0,BATCH_4,fastq,MFGHC0602_S2_L001_R1_001.fastq.gz,Raw sequencing data,110ca4864cf6938faca67567bebfb6cc,Raw,Reads,NA,NA,NA(raw data)
3,MFG_HC_0602,rep1,1,0,BATCH_4,fastq,MFGHC0602_S2_L001_R2_001.fastq.gz,Raw sequencing data,0dcc67217e43ab53bae0d0676f9bfe8b,Raw,Reads,NA,NA,NA(raw data)
4,MFG_PD_0009,rep1,1,0,BATCH_4,fastq,MFGPD0009_S3_L001_R1_001.fastq.gz,Raw sequencing data,a2608d0bd192333b0076d7091c1c50ea,Raw,Reads,NA,NA,NA(raw data)


In [25]:

# now fix the files and md5
BULK_FILES.head()

filenname_map = dict(zip(BULK_FILES['sample_id'], BULK_FILES['filename']))
md5_map = dict(zip(BULK_FILES['sample_id'], BULK_FILES['md5']))

DATA_['file_name'] = DATA_['sample_id'].map(filenname_map)
DATA_['file_MD5'] = DATA_['sample_id'].map(md5_map)


/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_36328/2870429344.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA_['file_name'] = DATA_['sample_id'].map(filenname_map)
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_36328/2870429344.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA_['file_MD5'] = DATA_['sample_id'].map(md5_map)


In [26]:
DATA_.columns

Index(['sample_id', 'replicate', 'replicate_count', 'repeated_sample', 'batch',
       'file_type', 'file_name', 'file_description', 'file_MD5', 'adjustment',
       'content', 'header', 'annotation', 'configuration_file'],
      dtype='object')

define "batch"

In [27]:
# set batch to "batch_x"
DATA_['batch'] = "batch_x"
# set batch to batch_x
SAMPLE_['batch'] = "batch_x"

/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_36328/501919316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA_['batch'] = "batch_x"
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_36328/501919316.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SAMPLE_['batch'] = "batch_x"


In [28]:
# pack the tables into a dictionary

v3_tables = {
    'STUDY': STUDY,
    'PROTOCOL': PROTOCOL,
    'SUBJECT': SUBJECT_,
    'SAMPLE': SAMPLE_,
    'DATA': DATA_,
    'CLINPATH': CLINPATH_,
    'PMDBS': PMDBS_,
    'ASSAY_RNAseq': ASSAY_RNAseq_,
    'CONDITION': CONDITION
}
CONDITION


,condition_id,intervention_name,intervention_id,protocol_id,intervention_aux_table
0,no_pd_nor_other_neurological_disorder,Case-Control,Control,NA,NA
1,other_neurological_disorder,Case-Control,Other,NA,NA
2,idiopathic_pd,Case-Control,Case,NA,NA


In [29]:
SAMPLE_['condition_id']

0     no_pd_nor_other_neurological_disorder
3               other_neurological_disorder
6                             idiopathic_pd
9                             idiopathic_pd
12                            idiopathic_pd
15                            idiopathic_pd
18                            idiopathic_pd
21    no_pd_nor_other_neurological_disorder
24    no_pd_nor_other_neurological_disorder
27    no_pd_nor_other_neurological_disorder
30    no_pd_nor_other_neurological_disorder
33    no_pd_nor_other_neurological_disorder
36              other_neurological_disorder
39    no_pd_nor_other_neurological_disorder
42    no_pd_nor_other_neurological_disorder
45                            idiopathic_pd
48                            idiopathic_pd
51                            idiopathic_pd
54                            idiopathic_pd
57                            idiopathic_pd
60                            idiopathic_pd
63                            idiopathic_pd
66                            id

In [30]:
for table,df in v3_tables.items():
    schema = CDE[CDE['Table'] == table]
    valid_fields = schema['Field'].unique()
    df_out = df[valid_fields]
    aux_fields = set(df.columns) - set(valid_fields)
    if aux_fields:
        df_aux = df[list(aux_fields)]
        df_aux.to_csv(og_path / f"{table}_auxiliary.csv", index=False)
        print(f"Saved {table}_auxiliary.csv")
    df_out.to_csv(v3_path / f"{table}.csv", index=False)

In [31]:
CDE = CDEv3
for table,df in v3_tables.items():
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()

recoding number_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **7 Fields with empty (NULL) values:**

	- submitter_email: 1/1 empty rows (REQUIRED)

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_ORCID: 1/1 empty rows (OPTIONAL)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- primary_diagnosis_text: 23/25 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding replicate_count as int
recoding repeated_sample as int
All required fields are present in *SAMPLE* table.


In [32]:
### Create metadata package

export_path = root_path / f"{team}"

create_upload_medadata_package(export_path, v3_tables)
v3_tables['SAMPLE']['condition_id']
v3_tables['CONDITION']

,condition_id,intervention_name,intervention_id,protocol_id,intervention_aux_table
0,no_pd_nor_other_neurological_disorder,Case-Control,Control,NA,NA
1,other_neurological_disorder,Case-Control,Other,NA,NA
2,idiopathic_pd,Case-Control,Case,NA,NA


In [120]:
SAMPLE_export.shape, SAMPLE_.shape, SAMPLE.shape, SAMPLE_out.shape

((25, 33), (25, 33), (75, 33), (25, 11))

In [121]:
report = ReportCollector(destination="log")

df = CLINPATH_.copy()
table_choice = "CLINPATH"
# perform the valadation

# specific_cde_df = CDE_df[CDE_df['Table'] == table_choice]
specific_cde_df = CDE_df[CDE_df['Table'].str.startswith(table_choice)]

retval = validate_table(df, table_choice, specific_cde_df, report)

report.print_log()

CLINPATH_export, CLINPATH_out, report = retval



validating subject_id
validating source_subject_id
validating duration_pmi
validating age_at_death
validating path_autopsy_dx_main
validating path_autopsy_second_dx
validating path_autopsy_third_dx
validating path_autopsy_fourth_dx
validating path_autopsy_fifth_dx
validating path_autopsy_sixth_dx
validating path_autopsy_seventh_dx
validating path_autopsy_eight_dx
validating path_year_death
validating cause_death
validating other_cause_death_1
validating other_cause_death_2
validating brain_weight
validating path_braak_nft
validating path_braak_asyn
validating path_cerad
validating path_thal
validating known_pathogenic_mutation
validating PD_pathogenic_mutation
validating path_mckeith
validating sn_neuronal_loss
validating path_infarcs
validating path_nia_ri
validating path_nia_aa_a
validating path_nia_aa_b
validating path_nia_aa_c
validating TDP43
validating arteriolosclerosis_severity_scale
validating amyloid_angiopathy_severity_scale
validating path_ad_level
validating dig_slide_avai

In [122]:
import numpy as np
df.replace({"":NULL, pd.NA:NULL, np.NaN:NULL}, inplace=True)


In [123]:
NULL

'NA'

In [124]:
SUBJECT_['age_at_onset'].values[0]

'NA'

In [125]:
report = ReportCollector(destination="log")

df = SUBJECT_.copy()
table_choice = "SUBJECT"
# perform the valadation

# specific_cde_df = CDE_df[CDE_df['Table'] == table_choice]
specific_cde_df = CDE_df[CDE_df['Table'].str.startswith(table_choice)]

retval = validate_table(df, table_choice, specific_cde_df, report)

report.print_log()

SUBJECT_export, SUBJECT_out, report = retval



validating subject_id
validating source_subject_id
validating biobank_name
validating organism
validating sex
validating age_at_collection
validating race
validating primary_diagnosis
validating primary_diagnosis_text
validating AMPPD_id
validating GP2_id
validating ethnicity
validating family_history
validating last_diagnosis
validating age_at_onset
validating age_at_diagnosis
validating first_motor_symptom
validating hx_dementia_mci
validating hx_melanoma
validating education_level
validating smoking_status
validating smoking_years
validating APOE_e4_status
validating cognitive_status
validating time_from_baseline
All required fields are present in *SUBJECT* table.
🚨⚠️❗ **12 Fields with empty (NULL) values:**

	- primary_diagnosis_text: 23/25 empty rows (OPTIONAL)

	- AMPPD_id: 25/25 empty rows (OPTIONAL)

	- GP2_id: 25/25 empty rows (OPTIONAL)

	- last_diagnosis: 25/25 empty rows (OPTIONAL)

	- age_at_onset: 25/25 empty rows (OPTIONAL)

	- age_at_diagnosis: 25/25 empty rows (OPTIONA

In [126]:
PROTOCOL

,sample_collection_summary,cell_extraction_summary,lib_prep_summary,data_processing_summary,github_url,protocols_io_DOI,other_reference
0,The samples were collected by the Banner Sun H...,See protocol.,Library preps were generated via the 5' 10x Ge...,We use cellranger count (6.0.1) to align and g...,private repo at https://github.com/TheGBLab/ASAP,https://dx.doi.org/10.17504/protocols.io.bzmgp43w,NA


In [127]:
STUDY['brain_regions'] = "middle frontal gyrus"
STUDY['team_dataset_id'] = "Human_bulkRNA_seq_PD_Senesence_Jose_Bras_Team_Lee"

Create ASSAY table:


In [128]:
STUDY['project_dataset']= 'Human bulkRNA-seq PD Senesence Jose Bras Team Lee'

In [129]:

metadata_path = Path.home() / ("Projects/ASAP/team-lee/metadata")

MFG_covar = pd.read_csv(f"{metadata_path}/MFG/covar.csv") # includes 'PMI' ?
MFG_cases = pd.read_csv(f"{metadata_path}/MFG/PD_ASAP_Sample_batch_information_banner_cases.csv").dropna(axis=0,how='all')
MFG_control = pd.read_csv(f"{metadata_path}/MFG/PD_ASAP_Sample_batch_information_banner_controls.csv")


In [130]:
### medial frontal gyrus samples
MFG_meta = pd.concat([MFG_cases, MFG_control], axis=0, ignore_index=True)
MFG_meta["GROUPcv"]= MFG_meta["PD"].apply(lambda x: "PD" if (x=="yes") else "HC")

# make a MERGE_ID column because the formatting is inconsistent
MFG_meta['MERGE_ID'] = "MFG_" + MFG_meta['GROUPcv'] +"_" + MFG_meta['CaseID'].str.replace('-','')
MFG_covar['MERGE_ID'] = MFG_covar['SAMPLE']
# the fastqs are in SEQ_ID 

# there's a bug in the meta table... skip for now
MFG_TABLE = pd.merge(MFG_covar, MFG_meta, on='MERGE_ID', how='inner')
MFG_TABLE['subdir']="MFG"



Adapt ASSAY_bulkRNAseq table from team Hardy.

In [131]:
metadata_path = Path.home() / "Projects/ASAP/meta-clean/data/teams/hardy/bulkRNAseq/metadata"

ASSAY_comp = pd.read_csv(metadata_path / "ASSAY_bulkRNAseq.csv")

In [132]:
ASSAY_comp

,sample_id,tissue,technology,omic,RIN,molecular_source,assay,sequencing_end,sequencing_length,sequencing_instrument,hemisphere,region_level_1,region_level_2,region_level_3,fragment_library_types
0,Hardy_001,Brain,Bulk,RNA,4.1,PolyA RNA,Other,Paired-end,150,Illumina NovaSeq 6000,Left,Frontal lobe,Middle frontal gyrus,Grey matter,ISR
1,Hardy_002,Brain,Bulk,RNA,3.1,PolyA RNA,Other,Paired-end,150,Illumina NovaSeq 6000,Left,Temporal lobe,Middle temporal gyrus,Grey matter,ISR
2,Hardy_003,Brain,Bulk,RNA,3.2,PolyA RNA,Other,Paired-end,150,Illumina NovaSeq 6000,Left,Parietal lobe,Inferior parietal lobule,Grey matter,ISR
3,Hardy_004,Brain,Bulk,RNA,3.9,PolyA RNA,Other,Paired-end,150,Illumina NovaSeq 6000,Left,Cingulate gyrus,Anterior cingulate gyrus,Grey matter,ISR
4,Hardy_005,Brain,Bulk,RNA,4.3,PolyA RNA,Other,Paired-end,150,Illumina NovaSeq 6000,Right,Frontal lobe,Middle frontal gyrus,Grey matter,ISR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,Hardy_284,Brain,Bulk,RNA,6.8,PolyA RNA,Other,Paired-end,150,Illumina NovaSeq 6000,Right,Cingulate gyrus,Anterior cingulate gyrus,Grey matter,ISR
284,Hardy_285,Brain,Bulk,RNA,6.4,PolyA RNA,Other,Paired-end,150,Illumina NovaSeq 6000,Right,Frontal lobe,Middle frontal gyrus,Grey matter,ISR
285,Hardy_286,Brain,Bulk,RNA,6.2,PolyA RNA,Other,Paired-end,150,Illumina NovaSeq 6000,Right,Temporal lobe,Middle temporal gyrus,Grey matter,ISR
286,Hardy_287,Brain,Bulk,RNA,5.6,PolyA RNA,Other,Paired-end,150,Illumina NovaSeq 6000,Right,Parietal lobe,Inferior parietal lobule,Grey matter,ISR


In [133]:
ASSAY = SAMPLE_ 



In [134]:
report = ReportCollector(destination="log")

df = ASSAY.copy()
table_choice = "ASSAY_bulkRNAseq"
# perform the valadation

# specific_cde_df = CDE_df[CDE_df['Table'] == table_choice]
specific_cde_df = CDE_df[CDE_df['Table'].str.startswith(table_choice)]

retval = validate_table(df, table_choice, specific_cde_df, report)

report.print_log()

ASSAY_export, ASSAY_out, report = retval

ASSAY_export

All required fields are present in *ASSAY_bulkRNAseq* table.
No empty entries (NULL) found .
No invalid entries found in Enum fields.



,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,MFG_HC_1225,HC_1225,12-25,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
1,MFG_HC_0602,HC_0602,06-02,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
2,MFG_PD_0009,PD_0009,00-09,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
3,MFG_PD_1921,PD_1921,19-21,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
4,MFG_PD_2058,PD_2058,20-58,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
5,MFG_PD_1441,PD_1441,14-41,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000383 (female),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
6,MFG_PD_1344,PD_1344,13-44,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000383 (female),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
7,MFG_HC_1939,HC_1939,19-39,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000383 (female),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
8,MFG_HC_1308,HC_1308,13-08,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
9,MFG_HC_1862,HC_1862,18-62,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA


In [135]:
export_path = Path.cwd() / "data/lee/bulk_rna_seq/metadata"

if not export_path.exists():
    export_path.mkdir(parents=True)


In [136]:
# fix the column order

# write the clean metadata
STUDY.to_csv(export_path / "STUDY.csv")
PROTOCOL.to_csv(export_path / "PROTOCOL.csv")
CLINPATH_export.to_csv(export_path / "CLINPATH.csv")
SAMPLE_export.to_csv(export_path / "SAMPLE.csv")
SUBJECT_export.to_csv(export_path / "SUBJECT.csv")

DATA_export.to_csv(export_path / "DATA.csv")

MFG_TABLE.to_csv(export_path / "aux_meta.csv")
# also writh them to clean...
# 
#  
ASSAY_export.to_csv(export_path / "ASSAY_bulkRNAseq.csv")

In [137]:
export_path / "ASSAY_bulkRNAseq.csv"

PosixPath('/Users/ergonyc/Projects/ASAP/harmonized-wf-dev/data/teams/lee/data/lee/bulk_rna_seq/metadata/ASSAY_bulkRNAseq.csv')

## Update the table to v2

In [126]:
SAMPLE.head()

,sample_id,source_sample_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,MFG_HC_1225,12-25,HC_1225,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,NaN,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,<NA>,NaN,
0,MFG_HC_1225,12-25,HC_1225,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,NaN,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,<NA>,NaN,
1,MFG_HC_0602,06-02,HC_0602,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,NaN,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,<NA>,NaN,
1,MFG_HC_0602,06-02,HC_0602,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,NaN,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,<NA>,NaN,
2,MFG_PD_0009,00-09,PD_0009,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,NaN,...,PATO:0000384 (male),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,<NA>,NaN,


In [127]:
from update_schema import update_tables_to_CDEv2

tables_path = Path.cwd() / "clean/team-Lee"


CDEv1 = pd.read_csv( Path.cwd() / "ASAP_CDE_v1.csv" )
CDEv2 = pd.read_csv( Path.cwd() / "ASAP_CDE_v2.csv" )


STUDYv2, PROTOCOLv2, SAMPLEv2, SUBJECTv2, CLINPATHv2, DATAv2 = update_tables_to_CDEv2(tables_path, CDEv1, CDEv2)


In [128]:

export_root = Path.cwd() / "clean/team-Lee/v2"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


In [129]:

STUDYv2.to_csv( export_root / "STUDY.csv")
PROTOCOLv2.to_csv(export_root / "PROTOCOL.csv")
SAMPLEv2.to_csv(export_root / "SAMPLE.csv")
SUBJECTv2.to_csv(export_root / "SUBJECT.csv")
CLINPATHv2.to_csv(export_root / "CLINPATH.csv")
DATAv2.to_csv(export_root / "DATA.csv")


In [130]:
export_root

PosixPath('/Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2')

Transfer cleaned metadata to raw buckets 




## Lee

In [131]:
# Lee
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/lee-credentials.json 


Activated service account credentials for: [raw-admin-lee@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [132]:

!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-lee/metadata/v2"


      6243  2023-11-29T22:26:06Z  gs://asap-raw-data-team-lee/metadata/v2/CLINPATH.csv#1701296766077305  metageneration=1
     24218  2023-11-29T22:26:06Z  gs://asap-raw-data-team-lee/metadata/v2/DATA.csv#1701296766689376  metageneration=1
       968  2023-11-29T22:26:07Z  gs://asap-raw-data-team-lee/metadata/v2/PROTOCOL.csv#1701296767085192  metageneration=1
     23092  2023-11-29T22:26:06Z  gs://asap-raw-data-team-lee/metadata/v2/SAMPLE.csv#1701296766486332  metageneration=1
      1054  2023-11-29T22:26:06Z  gs://asap-raw-data-team-lee/metadata/v2/STUDY.csv#1701296766878081  metageneration=1
      4277  2023-11-29T22:26:06Z  gs://asap-raw-data-team-lee/metadata/v2/SUBJECT.csv#1701296766288192  metageneration=1
TOTAL: 6 objects, 59852 bytes (58.45 KiB)


In [47]:
!gsutil -u dnastack-asap-parkinsons rm -r "gs://asap-raw-data-team-lee/metadata/v2/v2_20231128"

Removing gs://asap-raw-data-team-lee/metadata/v2/v2_20231128/CLINPATH.csv#1701213902124849...
Removing gs://asap-raw-data-team-lee/metadata/v2/v2_20231128/DATA.csv#1701213902800745...
Removing gs://asap-raw-data-team-lee/metadata/v2/v2_20231128/PROTOCOL.csv#1701213903243233...
Removing gs://asap-raw-data-team-lee/metadata/v2/v2_20231128/SAMPLE.csv#1701213902582568...
/ [4 objects]                                                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rm ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Removing gs://asap-raw-data-team-lee/metadata/v2/v2_20231128/STUDY.csv#1701213903050409...
Removing gs://asap-raw-data-team-lee/metadata/v2/v2_20231128/SUBJECT.csv#1701213902358354...
/ [6 objects]                                                                   
Operation completed over 6 objec

In [ ]:
Path.cwd()

PosixPath('/Users/ergonyc/Projects/ASAP/meta-clean')

In [133]:

!gsutil -u dnastack-asap-parkinsons cp -r "./clean/team-Lee/v2_20231130/*.csv"  "gs://asap-raw-data-team-lee/metadata/v2"

Copying file://./clean/team-Lee/v2_20231130/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/SUBJECT.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/SAMPLE.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/DATA.csv [Content-Type=text/csv]... 
- [4 files][ 56.5 KiB/ 56.5 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Lee/v2_20231130/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/PROTOCOL.csv [Content-Type=text/csv]...
\ [6 files][ 58.4 KiB/ 58.4 KiB]                                                
Operation completed over 6 objects/58.4 KiB.                                     


In [44]:
!gsutil -u dnastack-asap-parkinsons cp -r "./clean/team-Lee/*.csv"  "gs://asap-raw-data-team-lee/metadata/clean/"

Copying file://./clean/team-Lee/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/SUBJECT.csv [Content-Type=text/csv]...          
Copying file://./clean/team-Lee/auxiluary_metadata.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/SAMPLE.csv [Content-Type=text/csv]...           
\ [4 files][187.4 KiB/187.4 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Lee/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/auxilarry_metadata.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/PROTOCOL.csv [Content-Type=text/csv]...         
Copying file://./clean/team-Lee/CDE.csv [Content-Type=text/csv]...              
| [8 files][328.0 KiB/328.0 KiB]      

In [49]:
!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-lee/metadata/clean"


     37741  2023-11-28T21:58:33Z  gs://asap-raw-data-team-lee/metadata/clean/CDE.csv#1701208713502213  metageneration=1
     24241  2023-11-28T21:58:31Z  gs://asap-raw-data-team-lee/metadata/clean/CLINPATH.csv#1701208711158586  metageneration=1
       968  2023-11-28T21:58:33Z  gs://asap-raw-data-team-lee/metadata/clean/PROTOCOL.csv#1701208713057031  metageneration=1
     60412  2023-11-28T21:58:32Z  gs://asap-raw-data-team-lee/metadata/clean/SAMPLE.csv#1701208712230186  metageneration=1
       969  2023-11-28T21:58:32Z  gs://asap-raw-data-team-lee/metadata/clean/STUDY.csv#1701208712490381  metageneration=1
      3078  2023-11-28T21:58:31Z  gs://asap-raw-data-team-lee/metadata/clean/SUBJECT.csv#1701208711431682  metageneration=1
    104336  2023-11-28T21:58:32Z  gs://asap-raw-data-team-lee/metadata/clean/auxilarry_metadata.csv#1701208712852367  metageneration=1
    109904  2023-09-22T10:45:50Z  gs://asap-raw-data-team-lee/metadata/clean/auxiliary_metadata.csv#1695379550068707  metagene

### copy to workflow-dev bucket

First copy each set of metadata locally ...

In [135]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/wf-credentials.json

Activated service account credentials for: [admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [51]:
# !gsutil  ls -al "gs://asap-workflow-dev/CDE"

!gsutil  ls -al "gs://asap-workflow-dev/metadata/v2/lee"


      6243  2023-11-28T22:21:42Z  gs://asap-workflow-dev/metadata/v2/lee/CLINPATH.csv#1701210102669978  metageneration=1
     24218  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/DATA.csv#1701210103715354  metageneration=1
       968  2023-11-28T22:21:44Z  gs://asap-workflow-dev/metadata/v2/lee/PROTOCOL.csv#1701210104255218  metageneration=1
     23092  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/SAMPLE.csv#1701210103292269  metageneration=1
      1054  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/STUDY.csv#1701210103992608  metageneration=1
      4277  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/SUBJECT.csv#1701210103010639  metageneration=1
                                 gs://asap-workflow-dev/metadata/v2/lee/v2_20231128/
TOTAL: 6 objects, 59852 bytes (58.45 KiB)


In [136]:
!gsutil  cp -r "./clean/team-Lee/v2_20231130/*.csv" "gs://asap-workflow-dev/metadata/v2/lee"


Copying file://./clean/team-Lee/v2_20231130/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/SUBJECT.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/SAMPLE.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/DATA.csv [Content-Type=text/csv]... 
- [4 files][ 56.5 KiB/ 56.5 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Lee/v2_20231130/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/PROTOCOL.csv [Content-Type=text/csv]...
\ [6 files][ 58.4 KiB/ 58.4 KiB]                                                
Operation completed over 6 objects/58.4 KiB.                                     


In [56]:
!gsutil ls -al "gs://asap-workflow-dev/metadata/v2/lee"


      6243  2023-11-28T22:21:42Z  gs://asap-workflow-dev/metadata/v2/lee/CLINPATH.csv#1701210102669978  metageneration=1
     24218  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/DATA.csv#1701210103715354  metageneration=1
       968  2023-11-28T22:21:44Z  gs://asap-workflow-dev/metadata/v2/lee/PROTOCOL.csv#1701210104255218  metageneration=1
     23092  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/SAMPLE.csv#1701210103292269  metageneration=1
      1054  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/STUDY.csv#1701210103992608  metageneration=1
      4277  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/SUBJECT.csv#1701210103010639  metageneration=1
TOTAL: 6 objects, 59852 bytes (58.45 KiB)


## check file md5s

In [39]:
from utils.checksums import extract_md5_from_details, extract_md5_from_details2


In [36]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/lee-credentials.json  



Activated service account credentials for: [raw-admin-lee@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [37]:

# !gcloud storage hash "gs://asap-raw-data-team-lee/**/*.gz"  --skip-crc32c --hex  --billing-project dnastack-asap-parkinsons > hardy_hexhash.log

!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-lee/**/*.gz" > lee_hexhash.log


using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [40]:
bucket_files_md5 = extract_md5_from_details2("lee_hexhash.log")



checksum = DATAv2[['file_name','file_MD5']]
checksum['check1'] = checksum['file_MD5'].str.strip()
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)



/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_56157/1302936753.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check'] = checksum['file_name'].map(bucket_files_md5)


In [41]:
checksum[checksum.check1 != checksum.check2].file_name.to_list()
#empty means success!!


[]

In [42]:
checksum

,file_name,file_MD5,check
0,MFGHC1225_S9_L001_R1_001.fastq.gz,9977258e598d6a52130c29c71aef6925,9977258e598d6a52130c29c71aef6925
1,MFGHC1225_S9_L001_R2_001.fastq.gz,fe2cf93257801227b7072a4fb7d18792,fe2cf93257801227b7072a4fb7d18792
2,MFGHC0602_S2_L001_R1_001.fastq.gz,110ca4864cf6938faca67567bebfb6cc,110ca4864cf6938faca67567bebfb6cc
3,MFGHC0602_S2_L001_R2_001.fastq.gz,0dcc67217e43ab53bae0d0676f9bfe8b,0dcc67217e43ab53bae0d0676f9bfe8b
4,MFGPD0009_S3_L001_R1_001.fastq.gz,a2608d0bd192333b0076d7091c1c50ea,a2608d0bd192333b0076d7091c1c50ea
...,...,...,...
145,SN_1973_PD_S1_L000_R2_001.fastq.gz,53f6c6b4a00299fb41f32b34509835fa,53f6c6b4a00299fb41f32b34509835fa
146,SN_2005_PD_S1_L001_R1_001.fastq.gz,5bde8fdd8ab28e2c00e3ae327fecc80b,5bde8fdd8ab28e2c00e3ae327fecc80b
147,SN_2005_PD_S1_L001_R2_001.fastq.gz,14364dc52760fbd3ad08b5ca582d849b,14364dc52760fbd3ad08b5ca582d849b
148,SN_2038_PD_S1_L000_R1_001.fastq.gz,3215d4e4a68a85546183e820b74cac1f,3215d4e4a68a85546183e820b74cac1f
